<a href="https://colab.research.google.com/github/Abhishek92kumar/LJMU/blob/main/Publaynet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!git clone https://github.com/noba1anc3/Publaynet.git
%cd Publaynet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp310-cp310-linux_x86_64.whl size=44090 sha256=a3a66be7ab7df534caae6559ad0a2efd350f19d269566b48769134c81db1cdf3
  Stored in directory: /root/.cache/pip/wheels/70/83/31/975b737609aba39a4099d471d5684141c1fdc3404f97e7f68a
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.12.1 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
flax 0.6.9 requires jax>=0.4.2, but you have jax 0.3.25 which is incompatible.


In [2]:
!git clone https://github.com/facebookresearch/detectron2.git
%cd detectron2
!python -m pip install -e .
%cd ..


Cloning into 'detectron2'...
remote: Enumerating objects: 15054, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 15054 (delta 37), reused 61 (delta 35), pack-reused 14975
Receiving objects: 100% (15054/15054), 6.12 MiB | 19.12 MiB/s, done.
Resolving deltas: 100% (10900/10900), done.
/content/Publaynet/detectron2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/Publaynet/detectron2
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
!pip install webdataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 2.9 MB/s eta 0:00:00


In [5]:
import webdataset as wds
import pprint
import numpy as np
from itertools import islice
import torch
figsize(20, 10)
pp = pprint.PrettyPrinter(indent=2, compact=True, width=150, depth=2).pprint
cache_dir = None

# Loading PubLayNet from the Web

Datasets are accessible directly over the web. However, you can also store them locally, in network file systems, or in any object store.

In [ ]:
# urls = "http://storage.googleapis.com/lpr-publaynet/publaynet-train-{000000..000340}.tar"

In [6]:
urls =  "https://dax-cdn.cdn.appdomain.cloud/dax-publaynet/1.0.0/train-0.tar.gz"

Datasets are just `.tar` archives containing training files. For PubLayNet, each training sample consists of a `.png` file and a corresponding `.json` file.

In [7]:
!curl {urls} | tar xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12.6G  100 12.6G    0     0  43.6M      0  0:04:57  0:04:57 --:--:-- 52.4M


In [9]:
labels =  "https://dax-cdn.cdn.appdomain.cloud/dax-publaynet/1.0.0/labels.tar.gz"

In [8]:
!curl {labels} | tar xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: labels

gzip: stdin: unexpected end of file
tar: Child returned status 1
tar: Error is not recoverable: exiting now


In [ ]:
# !curl -s -L https://dax-cdn.cdn.appdomain.cloud/dax-publaynet/1.0.0/train-0.tar.gz | tar tvf - | sed 10q


We're instantiating a simple WebDataset without shuffling but with local caching. We'll later construct a more elaborate instance for training.

In [10]:
ds = wds.WebDataset(urls, shardshuffle=False, cache_dir=cache_dir).decode("rgb8")

WebDataset instances are just instances of PyTorch's IterableDataset

In [11]:
%mkdir data

%cp -rf /content/Publaynet/publaynet/train ./data/
%cp -rf /content/Publaynet/publaynet/train.json ./data/
%cp -rf /content/Publaynet/publaynet/test.json ./data/
%cp -rf /content/Publaynet/publaynet/val.json ./data/

%cd data
# !unzip train.zip
# !unzip val.zip
%cd ..

cp: cannot stat '/content/Publaynet/publaynet/train.json': No such file or directory
cp: cannot stat '/content/Publaynet/publaynet/test.json': No such file or directory
cp: cannot stat '/content/Publaynet/publaynet/val.json': No such file or directory
/content/Publaynet/data
/content/Publaynet


In [12]:
%cp -rf /content/Publaynet/data/train/test.json ./content/Publaynet/data/test
# %cp -rf /content/Publaynet/publaynet/val.json ./data/

cp: cannot stat '/content/Publaynet/data/train/test.json': No such file or directory


In [6]:
# %mkdir data

# %cp -rf ../drive/'My Drive'/train.zip ./data/
# %cp -rf ../drive/'My Drive'/val.zip ./data/

# %cd data
# !unzip train.zip
# !unzip val.zip
# %cd ..

cp: cannot stat '../drive/My Drive/train.zip': No such file or directory
cp: cannot stat '../drive/My Drive/val.zip': No such file or directory
/content/Publaynet/data
unzip:  cannot find or open train.zip, train.zip.zip or train.zip.ZIP.
unzip:  cannot find or open val.zip, val.zip.zip or val.zip.ZIP.
/content/Publaynet


In [13]:
!python train.py -f False

Traceback (most recent call last):
  File "/content/Publaynet/train.py", line 68, in <module>
    main(sys.argv[1:])
  File "/content/Publaynet/train.py", line 57, in main
    trainer = DefaultTrainer(cfg)
  File "/content/Publaynet/detectron2/detectron2/engine/defaults.py", line 376, in __init__
    model = self.build_model(cfg)
  File "/content/Publaynet/detectron2/detectron2/engine/defaults.py", line 514, in build_model
    model = build_model(cfg)
  File "/content/Publaynet/detectron2/detectron2/modeling/meta_arch/build.py", line 23, in build_model
    model.to(torch.device(cfg.MODEL.DEVICE))
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1145, in to
    return self._apply(convert)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 797, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 797, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.1

In [14]:
!python build.py -f true


In [15]:
# %rmdir --ignore-fail-on-non-empty
# 

rmdir: missing operand
Try 'rmdir --help' for more information.


In [45]:
# %rmdir --ignore-fail-on-non-empty /content/Publaynet/publaynet


In [16]:
!python predict.py -f true


Traceback (most recent call last):
  File "/content/Publaynet/predict.py", line 12, in <module>
    with open(valjsonPath, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: './data/test/test.json'


In [19]:
%ls /content/Publaynet/publaynet/train


Streaming output truncated to the last 5000 lines.
PMC4051442_00009.jpg  PMC5062254_00005.jpg  PMC5882787_00002.jpg
PMC4051548_00000.jpg  PMC5062254_00014.jpg  PMC5882791_00001.jpg
PMC4052176_00000.jpg  PMC5062254_00021.jpg  PMC5882866_00002.jpg
PMC4052430_00006.jpg  PMC5062254_00026.jpg  PMC5882961_00002.jpg
PMC4052452_00019.jpg  PMC5062564_00003.jpg  PMC5882979_00002.jpg
PMC4052478_00003.jpg  PMC5062692_00000.jpg  PMC5883194_00002.jpg
PMC4052517_00003.jpg  PMC5062694_00001.jpg  PMC5883212_00002.jpg
PMC4052517_00005.jpg  PMC5062697_00000.jpg  PMC5883240_00002.jpg
PMC4052541_00011.jpg  PMC5062706_00003.jpg  PMC5883268_00002.jpg
PMC4052628_00003.jpg  PMC5062708_00002.jpg  PMC5883286_00002.jpg
PMC4052662_00005.jpg  PMC5062733_00001.jpg  PMC5883325_00001.jpg
PMC4052662_00006.jpg  PMC5062828_00003.jpg  PMC5883336_00002.jpg
PMC4052683_00001.jpg  PMC5062873_00001.jpg  PMC5883337_00001.jpg
PMC4052683_00006.jpg  PMC5062873_00006.jpg  PMC5883361_00001.jpg
PMC4052785_00012.jpg  PMC5062873_00015.